In [1]:
def _hotfix10447_monkey():
    import energyplus.core
    DataExchange = energyplus.core.pyenergyplus.datatransfer.DataExchange

    if getattr(DataExchange, '_hotfix10447', False):
        return
    
    _get_api_data = DataExchange.get_api_data
    def _hotfix10447_get_api_data(self, state):
        return [
            DataExchange.APIDataExchangePoint(
                _what=datapoint.what,
                _name=datapoint.name, 
                _key=datapoint.type, 
                _type=datapoint.key,
            )
            for datapoint in (
                _get_api_data(self, state)
            )
        ]
    DataExchange.get_api_data = _hotfix10447_get_api_data
    setattr(DataExchange, '_hotfix10447', True)

_hotfix10447_monkey()

In [2]:
import logging as _logging_
_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

from energyplus.ooep.addons.progress import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

await simulator.awaitable.run(
    input=Simulator.InputSpecs(
        model=Model().loadf(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=Weather().open(_epds_.weathers / 'USA_FL_Tampa.Intl.AP.722110_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(report='.ooep-report-aaaaa'),
    options=Simulator.RuntimeOptions(
        design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

EnergyPlus Starting
EnergyPlus, Version 23.2.0-7636e6b3e9, YMD=2024.04.10 11:15
Converted input file format. Exiting.
EnergyPlus Run Time=00hr 00min  0.06sec


EnergyPlus Completed Successfully.


<Task pending name='Task-6' coro=<Engine.run() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:68>>

In [3]:
from energyplus.ooep import (
    Actuator,
    OutputVariable,
)

In [4]:
#%pip install pandas itables

import pandas as _pandas_
import itables as _itables_

# TODO plugin name: dataframe
class DataFrame(_pandas_.DataFrame):
    def _ipython_display_(self):
        return _itables_.show(self, classes='display')

Program terminated: EnergyPlus Terminated--Error(s) Detected.


In [5]:
list(simulator.variables.keys())

[WallClock.Ref(),
 Actuator.Ref(type='Weather Data', control_type='Outdoor Dry Bulb', key='Environment'),
 Actuator.Ref(type='Weather Data', control_type='Outdoor Dew Point', key='Environment'),
 Actuator.Ref(type='Weather Data', control_type='Outdoor Relative Humidity', key='Environment'),
 Actuator.Ref(type='Weather Data', control_type='Diffuse Solar', key='Environment'),
 Actuator.Ref(type='Weather Data', control_type='Direct Solar', key='Environment'),
 Actuator.Ref(type='Weather Data', control_type='Wind Speed', key='Environment'),
 Actuator.Ref(type='Weather Data', control_type='Wind Direction', key='Environment'),
 Actuator.Ref(type='Schedule:Compact', control_type='Schedule Value', key='ACTIVITY_SCH'),
 Actuator.Ref(type='Schedule:Compact', control_type='Schedule Value', key='AIR_VELO_SCH'),
 Actuator.Ref(type='Schedule:Compact', control_type='Schedule Value', key='ALWAYS_OFF'),
 Actuator.Ref(type='Schedule:Compact', control_type='Schedule Value', key='ALWAYS_ON'),
 Actuator.Re

In [6]:
simulator.variables.keys().group(type)[Actuator.Ref].group(lambda x: x.type)

{'Weather Data': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440ba390>, 'Schedule:Compact': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440ba690>, 'Schedule:Constant': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440ba990>, 'Material': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440bac90>, 'People': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440baf90>, 'Lights': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440bb290>, 'ElectricEquipment': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440bb4d0>, 'Surface': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440bb710>, 'Zone': <energyplus.ooep.components.variables.VariableManager.KeysView object at 0x7f62440bbad0>, 'Zone Infiltration': <energyplus.ooep.components.variables.V

In [7]:
DataFrame(
    simulator.variables.keys().group(type)[Actuator.Ref] #.table
)

In [8]:
DataFrame(
    simulator.variables.keys().group(type)[OutputVariable.Ref]
)
